In [37]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [38]:
import pandas as pd
from datetime import datetime

autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][0])
senha = str(autentica['senha'][0])

nome = 'CAFI'

data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

In [39]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import time

def abrir_suap():
    
    global nav
    nav = webdriver.Chrome()
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    
    # inserindo usuário
    nav.find_element(By.XPATH, '//*[@id="id_username"]').click()
    nav.find_element(By.XPATH, '//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element(By.XPATH, '//*[@id="id_password"]').click()
    nav.find_element(By.XPATH, '//*[@id="id_password"]').send_keys(senha)

    nav.find_element(By.XPATH, '//*[@id="ok"]').click()

In [40]:
# extraindo para lista
def table_to_list():
    tabela = []

    lista = nav.find_element(By.TAG_NAME, 'table')
    td = lista.find_elements(By.TAG_NAME, 'td')

    for i in td:
        tabela.append(i.text)

    return tabela

In [41]:
def build_df(lista):
    df_tabela = pd.DataFrame()


    tombos = []
    ele_des = []
    descricao = []
    cargas = []
    c_contabil = []
    salas = []
    dt_entrada = []
    fornecedores = []
    valores = []

    #  remove o primeiro elemento ta tabela, informação quebrada    
    for excluir in range(0, len(lista), 1201):
        lista.pop(excluir)

    for tombo in range(1, len(lista)+1, 12):
        if lista[tombo] == '':
            lista.pop(tombo)

    # listas para montar o data frame
    for tombo in range(1, len(lista), 12):
        tombos.append(lista[tombo])

    for ed in range(2, len(lista), 12):
        ele_des.append(lista[ed])

    for des in range(3, len(lista), 12):
        descricao.append(lista[des])

    for carga in range(4, len(lista), 12):
        cargas.append(lista[carga])

    for c in range(5, len(lista), 12):
        c_contabil.append(lista[c])

    for sala in range(6, len(lista), 12):
        salas.append(lista[sala])

    for data in range(8, len(lista), 12):
        dt_entrada.append(lista[data])
    
    for f in range(10, len(lista), 12):
        fornecedores.append(lista[f])

    for valor in range(11, len(lista), 12):
        valores.append(lista[valor])



    #  montagem do data frame
    df_tabela['tombo'] = tombos
    df_tabela['tombo'] = df_tabela['tombo'].astype(str).str[:-6]

    df_tabela['ed'] = ele_des

    df_tabela['descricao'] = descricao

    df_tabela['carga'] = cargas
    df_tabela['sala'] = salas

    df_tabela['c_contabil'] = c_contabil

    df_tabela['dt_entrada'] = dt_entrada
#     df_tabela['dt_entrada'] = pd.to_datetime(df_tabela['dt_entrada'], infer_datetime_format=True)

    df_tabela['fornecedor'] = fornecedores
    
    df_tabela['valor'] = valores
    df_tabela['valor'] = df_tabela['valor'].str.replace('.','',regex=True)
    df_tabela['valor'] = df_tabela['valor'].str.replace(',','.',regex=True)
    df_tabela['valor'] = pd.to_numeric(df_tabela['valor'])
    df_tabela['valor'] = df_tabela['valor'].round(2)

    return df_tabela

In [42]:
def pagination():
    
    abrir_suap()
    print(f'Iniciou as: {datetime.now()}')
    pag = 1
    saida = False
    complete_df = pd.DataFrame()
    
    try:
        while saida is False:

            nav.get(f'https://suap.ifpe.edu.br/patrimonio/inventario_busca/?page={pag}&status=&empenho=&empenho=&data_final=&sala=&numero_serie=&numero=&rotulos=&data_inicial=&estado_conservacao=6&elemento_despesa=&valor=&responsavel=&responsavel=&contabil=&fornecedor=&fornecedor=&setor_responsavel=&campus=12&descricao=')
            
            celula = nav.find_element(By.CLASS_NAME, 'row1')

            if celula:
                suap_list = table_to_list()
                suap_df = build_df(suap_list)
                complete_df = pd.concat([complete_df, suap_df], ignore_index= True)
                pag += 1

            else:
                saida = True
    
    except NoSuchElementException:
        print('acabaram as páginas')
        
    print(f'Terminou as: {datetime.now()}')
#     display(complete_df)
    complete_df.to_excel(f'extracoes_suap/{data_e_hora}_{nome}_relatorio_patrimonio.xlsx', index=False)
    
    nav.quit()
    
pagination()

Iniciou as: 2022-11-10 10:13:36.433373
acabaram as páginas
Terminou as: 2022-11-10 11:35:14.581835


,tombo,ed,descricao,carga,sala,c_contabil,dt_entrada,fornecedor,valor
0,33,449052-42,182CREC - BANCO P/DESENHO (FIXO COM 75CM DE AL...,VANESSA RAMOS ALVES (CAFI AFI-DEN),,RECIFE,16/06/2010,28/12/2010,0.01
1,3842,449052-42,14471CREC - ARMARIO (DE A O PARA ACESSORIOS DE...,Carlos Eduardo Cabral Rodrigues (CAFI AFI-CCBEC),,RECIFE,16/06/2010,28/12/2010,0.01
2,16809,449052-10,51132CREC - TABELA DE BASQUETEBOL MOVEL (PAR) ...,Ezenildo Emanuel de Lima (CAFI AFI-CODAGR),,AFOGADOS DA INGAZEIRA,16/06/2010,28/12/2010,12698.00
3,17274,449052-35,51615CREC - MICROCOMPUTADOR PESSOAL (NOTEBOOK ...,Ezenildo Emanuel de Lima (CAFI AFI-CODAGR),ANEXO DO ALMOXARIFADO - BLOCO C: LABORATÓRIOS ...,RECIFE,16/06/2010,28/12/2010,4553.00
4,27262,449052-52,"43983CPES - VEICULO (FORD,FIESTA1.6 04P.5PAS.A...",Ezenildo Emanuel de Lima (CAFI AFI-CODAGR),,AFOGADOS DA INGAZEIRA,16/06/2010,28/12/2010,28397.40
...,...,...,...,...,...,...,...,...,...
10002,269539,44905235,KIT ARDUÍNO ACESSÓRIOS PARA ESTUDO/TREINAMENTO...,ANDREA FERREIRA DACAL (CAFI AFI-DEN),,REITORIA,15/03/2022,08/04/2022,470.00
10003,269540,44905235,KIT ARDUÍNO ACESSÓRIOS PARA ESTUDO/TREINAMENTO...,ANDREA FERREIRA DACAL (CAFI AFI-DEN),,REITORIA,15/03/2022,08/04/2022,470.00
10004,269541,44905235,KIT ARDUÍNO ACESSÓRIOS PARA ESTUDO/TREINAMENTO...,ANDREA FERREIRA DACAL (CAFI AFI-DEN),,REITORIA,15/03/2022,08/04/2022,470.00
10005,269915,449052-45,IMPRESSORA 3D CREALITY CR 10 V2; Serial P202J0...,ANDREA FERREIRA DACAL (CAFI AFI-DEN),,REITORIA,29/03/2022,08/04/2022,4652.00
